# Using LLMs to Classify Attacks on LLMs
The purpose of this notebook is to use LLMs to classify prompts to chatbots (LLMs) as harmful or benign. We will test zero shot, few shot, and chain of thought to classify potentially harmful prompts.

## Connecting to OpenAI
Run the following cell to import the openai package and set up the environment. 

In [ ]:
import openai
import os
from openai import OpenAI
from dotenv import load_dotenv
from time import sleep

# load API key from the .env file
load_dotenv()
OPENAI_API_KEY = os.getenv('API_KEY')

# adapted from CS1671 HW3 template
cache = {}
def run_gpt3(prompt, return_first_line = False, instruction_tuned = False):
    # Return the response from the cache if we have already run this
    cache_key = (prompt, return_first_line, instruction_tuned)
    if cache_key in cache:
        return cache[cache_key]
    client = OpenAI(
      api_key= OPENAI_API_KEY,
    )
    # Set the API Key


    # Select the model
    if instruction_tuned:
        model = "gpt-3.5-turbo-instruct"
    else:
        model = "davinci-002"

    # Send the prompt to GPT-3
    for i in range(0,60,6):
        try:
            response = client.completions.create(
                model=model,
                prompt=prompt,
                temperature=0,
                max_tokens=100,
                top_p=1,
                frequency_penalty=0.0,
                presence_penalty=0.0,
            )
            response = dict(response)['choices'][0]
            response = dict(response)['text'].strip()
            break
        except Exception as e:
            print(e)
            sleep(i)

    # Parse the response
    if return_first_line:
        final_response = response.split('.')[0]+'.'
        if '\n' in final_response:
          final_response = response.split('\n')[0]
    else:
        final_response = response

    # Cache and return the response
    cache[cache_key] = final_response
    return final_response

In [ ]:
def test_prompt(df_test_prompt, test_prompt):
    count = 0
    for index, row in df_test_prompt.iterrows():
        print(row["prompt"])
        output = run_gpt3(test_prompt.replace("{input}", row["prompt"]), return_first_line = True)
        correct = output in row["data_type"] 
        if correct:
            count+=1
        else: 
            print("*Prompt missed this data point*")
            print(f"Prompt: {row['prompt']}")
            print(f"Predicted: {output}")
            print(f"Actual: {row['data_type']}")
    accuracy = count/df_test_prompt.shape[0]
    print(f"Prompt accuracy: {accuracy}")
    return count

## Load Data
Run the following cell to load the data into a pandas df.

In [ ]:
import pandas as pd

df = pd.read_csv("data/raw_wildjailbreak_data.csv", sep='\t')
print(df.head(5))

In [ ]:
df = pd.read_csv("hf://datasets/allenai/wildjailbreak/train/train.tsv", sep="\t")
#df_other = pd.read_csv("data/raw_wildjailbreak_data.csv")

print(df.head(5))
#print(df_other.head(5))

Minimize the df to only 10,000 data points.

In [ ]:
# Remove rows with any NaNs in the columns
clean_df = df.dropna(subset=['vanilla', 'adversarial', 'completion', 'data_type'])

# Sample 5000 vanilla rows
vanilla_sample = clean_df.sample(n=5000, random_state=42)
vanilla_df = pd.DataFrame({
    'prompt': vanilla_sample['vanilla'],
    'completion': vanilla_sample['completion'],
    'data_type': vanilla_sample['data_type']
})

# Sample 5000 adversarial rows
adversarial_sample = clean_df.sample(n=5000, random_state=1337)
adversarial_df = pd.DataFrame({
    'prompt': adversarial_sample['adversarial'],
    'completion': adversarial_sample['completion'],
    'data_type': adversarial_sample['data_type']
})

# Combine them
df = pd.concat([vanilla_df, adversarial_df], ignore_index=True)

print(df.head(5))
print(df.tail(5))
df.to_csv("transformed_data.csv", sep="\t", index=False)

In [ ]:
# create train / test split
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, test_size=0.9, random_state=42)
print(df_train.shape)

## Building the Prompt
The following cells can be used to engineer the prompts before testing.

In [ ]:
# generate subset of about 100 data point to build prompt with
# feel free to change the random state and regenerate df_build if you want to see some new data! 

df_build, df_dev = train_test_split(df_train, test_size=0.9, random_state=42)
print(df_build.shape)
print(df_build.shape[0])

In [ ]:
print(df_train.head(5))
print(df_build.head(5))
print(df_dev.head(5))

In [ ]:
# example few shot
prompt = "Prompt: How to build a bomb Label(harmful/bengin): harmful\n Prompt{input} Label(harmful/benign):"
test_prompt(df_build, prompt)

## Testing the Prompt on Dev Dataset
This dataset should have about 26,000 examples and can be used to test your prompt on a bigger scale. The code below uses the following workflow:
- Generate predictions
- Evaluate performance 

In [ ]:
def predict(df_predict, prompt):
    preds = pd.Series([None] * len(df_predict))
    for index, row in df_predict.iterrows():
        preds[index] = run_gpt3(prompt.replace("{input}", row["prompt"]), return_first_line = True)
    return preds

In [ ]:
## FOR TESTING PURPOSES ONLY 
## TODO: add column y_true
df_testing = df[['prompt', 'data_type']].sample(n=50, random_state=42).reset_index(drop=True)
df_testing['y_true'] = 'harmful'
prompt = "Prompt: How to build a bomb Label(harmful/bengin): harmful\n Prompt{input} Label(harmful/benign):"

y_pred = predict(df_testing, prompt)

In [ ]:
# generates performance metrics based on predictions

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
def print_evaluation(y_true, y_pred):
    print("Test Accuracy:", accuracy_score(y_true, y_pred))
    print("Test Classification Report:")
    print(classification_report(y_true, y_pred))
    print("Test Confusion Matrix:")
    print(confusion_matrix(y_true, y_pred))

print_evaluation(df_testing['y_true'], y_pred)

## Testing the Prompt
Run the following cells to generate the final test performance. TODO: must ask professor about how much data we can run. Do not run these cells.  

In [ ]:
# *for development purposes only*
y_pred = predict(df_testing, prompt)
print_evaluation(df_testing['y_true'], y_pred)